# preprocess2
> 결측치 처리

 - 1. 현재연면적 : **변수 제거**
 - 2. 빌딩비고, 거래비고 : **결측치 대체** : 일반
 - 3. 지상층, 지하층, 준공년도 : **결측행 제거**
 - 4. APT매매가격지수 : **대체 (이후 값으로 대체)**
 - 5. 업무용_실거래가_연면적당평균 : **대체 (보간법)**
 - 6. 주거용_실거래가_연면적당평균 : **대체 (보간법)**  
 - 7. 주거용_건물용도별_실거래가_연면적당평균 : **대체 (보간법)**  
   : 연립다세대, 단독다가구, 오피스텔, 아파트
 - 8. 상업용임대동향 지표 (오피스_) : **대체**  
   : 평균연면적(㎡),공실률(%),전환율(%),임대료(천원/㎡), 임대수입(%),기타수입(%),영업경비(%),  
  순영업소득(%),순영업소득(천원/㎡), 소득수익률(%),자본수익률(%),투자수익률(%)
 - +건물연식 : 2024-준공년도
 - +변수명 정제 : 단위-특수기호 제거

In [1]:
## 라이브러리 호출
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import re

# 한글폰트 설정
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False #음수에 (-)표현
pd.set_option('display.max_columns',None)

# Warning 방지
import warnings
warnings.filterwarnings('ignore')

# 0. processed1_merged
1차 전처리 데이터셋 결측치

In [2]:
# processed1 불러오기
df0 = pd.read_csv('../data/processed1_merged.csv', encoding='cp949')
print('Shape of processed1_merged dataset:', df0.shape)

# 각 열의 결측치 수 계산
na_counts = df0.iloc[:,1:].isnull().sum()
na_df = pd.DataFrame({'Column': na_counts.index, 'NA_Count': na_counts.values})
num_rows = (len(na_df) +3- 2) // 3  # 전체 행 수 계산
na_df_gathered = pd.DataFrame(columns=['Column1', 'NA_Count1', 'Column2', 'NA_Count2', 'Column3', 'NA_Count3'])
for i in range(num_rows):
    row_data = []
    for j in range(3):
        index = i + j * num_rows
        if index < len(na_df):
            row_data.append(na_df.iloc[index, 0])  # Column
            row_data.append(na_df.iloc[index, 1])  # NA_Count
        else:
            row_data.extend([None, None])  # 남는 부분은 None으로 채우기
    na_df_gathered.loc[i] = row_data
na_df_gathered

Shape of processed1_merged dataset: (972, 43)


,Column1,NA_Count1,Column2,NA_Count2,Column3,NA_Count3
0,거래분기,0,권역,0,오피스텔_실거래가_건물면적당평균(만원/㎡),266
1,시,0,기준금리(%),0,주거용_실거래가_건물면적당평균(만원/㎡),181
2,구,0,APT매매가격지수,34,오피스_평균연면적(㎡),49
3,동,0,오피스매매가격지수,0,오피스_공실률(%),49
4,거래당시연면적,0,소비자물가지수,0,오피스_전환율(%),49
5,거래당시토지면적,0,건설_소계_건설공사비지수,0,오피스_임대료(천원/㎡),49
6,현재연면적,936,주거용건물_건설공사비지수,0,오피스_임대수입(%),49
7,빌딩비고,847,비주거용건물_건설공사비지수,0,오피스_기타수입(%),49
8,거래비고,952,건축보수_건설공사비지수,0,오피스_영업경비(%),49
9,거래가,0,표준지공시지가평균(원/㎡),0,오피스_순영업소득(%),49


# 1. 현재연면적
총 972rows 중 936row가 NA
> 결측치 처리 : 변수 제거

In [3]:
df1_final = df0.drop(columns='현재연면적')

print('Shape of dataset before step1:', df0.shape)
print('Shape of dataset after step1:', df1_final.shape)

Shape of dataset before step1: (972, 43)
Shape of dataset after step1: (972, 42)


# 2. 빌딩비고, 거래비고
> 결측치 처리 : '일반'으로 대체

In [4]:
df2_final = df1_final.copy()
df2_final['빌딩비고'] = df1_final['빌딩비고'].fillna('일반')
df2_final['거래비고'] = df2_final['거래비고'].fillna('일반거래')
print(df2_final[['빌딩비고','거래비고']].value_counts())
print(df2_final[['빌딩비고','거래비고']].isna().sum())

빌딩비고  거래비고       
일반    일반거래           827
멸실    일반거래            89
부분멸실  일반거래            24
증축    일반거래            12
일반    건물중오피스부분만거래     11
      다른필지건물포함거래       9
Name: count, dtype: int64
빌딩비고    0
거래비고    0
dtype: int64


# 3. 지상층, 지하층, 준공년도
결측행
- 지상층, 지하층 : 2rows NA
- 준공년도 : 3rows NA

> 결측치 처리 : 행 제거 : 4rows

In [5]:
# 결측행 제거
df3_final = df2_final.dropna(subset=['지상층','지하층','준공년도']).reset_index(drop=True)

print('Shape of dataset before step3:', df2_final.shape)
print('Shape of dataset after step3:', df3_final.shape)

Shape of dataset before step3: (972, 42)
Shape of dataset after step3: (968, 42)


# 4. APT매매가격지수
결측행 : 34rows
> 결측치 처리 : 대체 (보외법)

In [ ]:
df0[df0['APT매매가격지수'].isna()][['거래년도','거래분기']].value_counts()
# NA: 2014년 1,2분기 데이터 -> 보외법으로 대체

거래년도  거래분기
2014  2Q      19
      1Q      15
Name: count, dtype: int64

In [7]:
# 데이터프레임 복사
df4 = df3_final.copy()

# 거래분기를 월로 변환 (1Q -> 02, 2Q -> 05 등 각 분기의 중앙 달을 사용)
quarter_to_month = {'1Q': '02', '2Q': '05', '3Q': '08', '4Q': '11'}
df4['거래월'] = df4['거래분기'].map(quarter_to_month)

# 거래년도와 거래월을 결합한 '거래시점' 열 생성 (YYYY-MM 형식)
df4['거래시점'] = pd.to_datetime(df4['거래년도'].astype(str) + '-' + df4['거래월'])

# '구'별로 그룹화하고, 거래시점을 기준으로 정렬한 후 보외법 적용
df4['APT매매가격지수'] = df4.groupby('구', group_keys=False).apply(
    lambda group: group.sort_values('거래시점')['APT매매가격지수'].interpolate(method='linear', limit_direction='both')
)
#중복 행 삭제
df4.drop_duplicates(inplace=True)
df4 = df4.loc[:,['거래년도','거래분기','동','APT매매가격지수']]

df4['APT매매가격지수'].isna().sum()

0

In [8]:
print(df3_final[['거래년도','거래분기','동']].equals(df4.iloc[:, :3]))
df4_final = df3_final.copy()
df4_final['APT매매가격지수'] = df4['APT매매가격지수']
df4_final['APT매매가격지수'].isna().sum()

True


0

# 5. 업무용_실거래가_연면적당평균(만원/㎡)
결측행 : 223rows
> 결측치 처리 : 대체 (보간법)

In [9]:
# 1차 전처리 후, merge 이전 파일
df5 = pd.read_csv('../data/processed1_업무용_실거래가_연면적당평균.csv', encoding='cp949')
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4049 entries, 0 to 4048
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   거래년도                   4049 non-null   int64  
 1   거래분기                   4049 non-null   object 
 2   시                      4049 non-null   object 
 3   구                      4049 non-null   object 
 4   동                      4049 non-null   object 
 5   업무용_실거래가_연면적당평균(만원/㎡)  4049 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 189.9+ KB


raw_data에 없는 거래년도,거래분기,구,동 탐색 후 생성

In [10]:
# df5['동']의 고유값
unique_df = df5['동'].unique()

# df4_final['동']의 고유값
unique_df_merge = df4_final['동'].unique()

# df4_final['동']에 속하는 값들이 df5['동']에 모두 포함되는지 확인
is_included = np.isin(unique_df_merge, unique_df)

# 결과 출력
if all(is_included):
    print('포함')
else:
    print('포함X 동 존재')
    missing_values = unique_df_merge[~is_included]
    print("포함되지 않은 값들:", missing_values)
    # raw_data에서 값이 하나도 없는 '동'

# 구-동 매핑 딕셔너리 생성
dong_gu_mapping = {
    '염곡동': '서초구',
    '무교동': '중구',
    '보문동5가' : '성북구'
}

# 추가할 행 생성
additional_rows = []
for dong, gu in dong_gu_mapping.items():
    for year in range(2014, 2025):
        for quarter in ['1Q', '2Q', '3Q', '4Q']:
            additional_rows.append({
                '거래년도': year,
                '거래분기': quarter,
                '구': gu,
                '동': dong,
                '업무용_실거래가_연면적당평균(만원/㎡)': None  # 혹시 실거래가가 없다면 None으로 설정
            })
# 추가할 데이터프레임 생성
additional_df = pd.DataFrame(additional_rows)
print('Shape before add:', df5.shape)

# 기존 df와 추가된 df 결합
df5 = pd.concat([df5, additional_df], axis=0)
print('Shape after add:', df5.shape)
df5['동'].value_counts().head(3) 
# 3개의 '동' 생성, 132개 행 증가

포함X 동 존재
포함되지 않은 값들: ['염곡동' '보문동5가' '무교동']
Shape before add: (4049, 6)
Shape after add: (4181, 6)


동
보문동5가    44
무교동      44
염곡동      44
Name: count, dtype: int64

In [11]:
# 1. 전체 조합 생성 (거래년도, 거래분기, 동, 구)
years = range(2014, 2025)
quarters = ['1Q', '2Q', '3Q', '4Q']
all_periods = [(year, quarter) for year in years for quarter in quarters]
dong_list = df5['동'].unique()
combination = []
for (year, quarter) in all_periods:
    for dong in dong_list:
        gu = df5.loc[df5['동'] == dong, '구'].values[0]  # 동에 해당하는 구를 직접 가져옴
        combination.append((year, quarter, dong, gu))
# 전체 조합의 개수
print(f"Total combinations: {len(combination)}")

# 2. 기존 데이터셋에서 조합을 찾아서 없는 조합을 찾기
existing_combinations = df5[['거래년도', '거래분기', '동', '구']].apply(tuple, axis=1).tolist() #기존데이터
missing_combinations = [combo for combo in combination if combo not in existing_combinations] #없는 조합
print(f"Existing combinations: {len(existing_combinations)}")
print(f"Missing combinations: {len(missing_combinations)}")

# 누락된 조합의 중복 체크
missing_duplicates = len(missing_combinations) - len(set(missing_combinations))
print(f"Number of duplicate missing combinations: {missing_duplicates}")

# 3. 새로운 조합을 데이터프레임으로 만들고 NA 값으로 채우기
new_rows = pd.DataFrame(missing_combinations, columns=['거래년도', '거래분기', '동', '구'])

# 기존 데이터프레임의 업무용 실거래가 열은 그대로 두고 새로 생성된 열에 NA 할당
new_rows['업무용_실거래가_연면적당평균(만원/㎡)'] = pd.NA

# 최종 결과물 생성: 기존 데이터프레임과 새로 생성된 행 결합
df5_updated = pd.concat([df5, new_rows], ignore_index=True)
df5_updated.info()

Total combinations: 15576
Existing combinations: 4181
Missing combinations: 11399
Number of duplicate missing combinations: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15580 entries, 0 to 15579
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   거래년도                   15580 non-null  int64  
 1   거래분기                   15580 non-null  object 
 2   시                      4049 non-null   object 
 3   구                      15580 non-null  object 
 4   동                      15580 non-null  object 
 5   업무용_실거래가_연면적당평균(만원/㎡)  4049 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 730.4+ KB


In [12]:
# 거래날짜 임의로 생성
def convert_to_date(row):
    year = row['거래년도']
    quarter = row['거래분기']
    month = (int(quarter[0]) - 1) * 3 + 1  # 1Q -> 1, 2Q -> 4, 3Q -> 7, 4Q -> 10
    return pd.Timestamp(year=year, month=month, day=1)

df5_updated['거래날짜'] = df5_updated.apply(convert_to_date, axis=1)
df5_updated['거래날짜'] = pd.to_datetime(df5_updated['거래날짜'])

df5_interpolated = df5_updated.copy()

df5_interpolated['거래날짜'] = pd.to_datetime(df5_interpolated['거래년도'].astype(str) + '-01-01')
df5_interpolated.set_index('거래날짜', inplace=True)
df5_interpolated.head()

,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
거래날짜,,,,,,
2014-01-01,2014,1Q,성남,분당구,구미동,354.012138
2014-01-01,2014,1Q,성남,분당구,궁내동,268.600591
2014-01-01,2014,1Q,성남,분당구,금곡동,268.101328
2014-01-01,2014,1Q,성남,분당구,삼평동,436.963754
2014-01-01,2014,1Q,성남,분당구,서현동,610.069339


**결측값 대체** 
- 동별 보간: raw_data 동별 값 2개 이상 존재  
- 구별 보간: raw_data 동별 값  or 존재X 

1. 시간 보간  
 : 결측치 아닌행 3개 이상 & 양끝값 존재
2. 선형 보간  
  : 결측치 아닌행 2개 이상

In [13]:
## 동별 보간

# 1단계: 동 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
def time_interpolate(group):
    return group.interpolate(method='time')

# 3개 이상인 그룹에 대해 시간 보간 수행
df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'] = (
    df5_interpolated.groupby(['동'])['업무용_실거래가_연면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 2단계: 남은 결측치 중 동 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
def linear_interpolate_if_enough(group):
    if len(group) == 2:
        return group.interpolate(method='linear')
    return group

# 2개 이상인 그룹에 대해 선형 보간 수행
df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'] = (
    df5_interpolated.groupby(['동'])['업무용_실거래가_연면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 결측치 확인
print("업무용 실거래가 결측치 수 (전체):", df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'].isnull().sum())

# 인덱스를 원래 형태로 되돌리기
df5_interpolated.reset_index(inplace=True)


업무용 실거래가 결측치 수 (전체): 132


In [ ]:
## 구별 보간

df5_interpolated.set_index('거래날짜', inplace=True)

# 타겟 동 목록 정의
target_dongs = ['염곡동', '무교동']  # 타겟 동 목록

# 1단계: 동, 구별, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'] = (
    df5_interpolated.groupby(['구'])['업무용_실거래가_연면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 인덱스를 원래대로 되돌리기
df5_interpolated.reset_index(inplace=True)

# 2단계: 남은 결측치 중 구별, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'] = (
    df5_interpolated.groupby(['구'])['업무용_실거래가_연면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 전체 결측치 확인
print("업무용 실거래가 결측치 수:", df5_interpolated['업무용_실거래가_연면적당평균(만원/㎡)'].isnull().sum())


업무용 실거래가 결측치 수: 0


In [ ]:
df5_col = pd.merge(df4_final, df5_interpolated, on=['거래년도','거래분기','구','동'], how='left').iloc[:,-1]
df5_final = df4_final.copy()
df5_final['업무용_실거래가_연면적당평균(만원/㎡)'] = df5_col
df5_final['업무용_실거래가_연면적당평균(만원/㎡)'].isna().sum()

0

# 6. 주거용_건물용도별_실거래가_연면적당평균(만원/㎡)
단독다가구, 연립다세대, 아파트, 오피스텔
> 결측치 처리 : 대체 (보간법)

In [ ]:
# 1차 전처리 후, merge 이전 파일
df6 = pd.read_csv('../data/processed1_주거용_실거래가_연면적당평균_건물용도별.csv', encoding='cp949')
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43225 entries, 0 to 43224
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   거래년도                    43225 non-null  int64  
 1   거래분기                    43225 non-null  object 
 2   구                       43225 non-null  object 
 3   동                       43225 non-null  object 
 4   건물용도                    43225 non-null  object 
 5   주거용_실거래가_건물면적당평균(만원/㎡)  43225 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 2.0+ MB


In [17]:
# df6['동']의 고유값
unique_df = df6['동'].unique() #428

# df5_final['동']의 고유값
unique_df_merge = df5_final['동'].unique() #188

# df5_final['동']에 속하는 값들이 df7['동']에 모두 포함되는지 확인
is_included = np.isin(unique_df_merge, unique_df)

# 결과 출력
if all(is_included):
    print('포함')
else:
    print('포함X 동 존재')
    missing_values = unique_df_merge[~is_included]
    print("포함되지 않은 값들:", missing_values)
    # raw_data에서 값이 하나도 없는 '동'

# 동 list
target_dongs = [
    '청진동', '소공동', '서린동', '무교동', '수하동', '종로2가', 
    '태평로2가', '충정로1가', '도렴동', '봉래동1가', '을지로1가', 
    '공평동', '중학동', '남대문로1가', '남대문로2가',
    '야탑동', '수내동', '서현동', '구미동', '금곡동', 
    '백현동', '정자동', '운중동', '삼평동'
] #24

# 동에 해당하는 구 가져오기
target_mapping = df5_final[df5_final['동'].isin(target_dongs)][['동', '구']].drop_duplicates()
print("타겟 동과 구 매핑 결과:")
print(target_mapping)

# 추가할 행 생성
additional_rows = []
for index, row in target_mapping.iterrows():  # target_mapping을 DataFrame으로 유지
    dong = row['동']
    gu = row['구']
    for year in range(2014, 2025):
        for quarter in ['1Q', '2Q', '3Q', '4Q']:
            for building in ['단독다가구','아파트','연립다세대','오피스텔']:
                additional_rows.append({
                    '거래년도': year,
                    '거래분기': quarter,
                    '구': gu,
                    '동': dong,
                    '건물용도' : building,
                    '주거용_실거래가_건물면적당평균(만원/㎡)': None  # 실거래가가 없다면 None으로 설정
                })

# 추가할 데이터프레임 생성
additional_df = pd.DataFrame(additional_rows)
print('Shape before add:', df6.shape)

# 기존 df와 추가된 df 결합
df6 = pd.concat([df6, additional_df], ignore_index=True)
print('Shape after add:', df6.shape)

포함X 동 존재
포함되지 않은 값들: ['청진동' '소공동' '야탑동' '수내동' '서현동' '서린동' '구미동' '무교동' '수하동' '금곡동' '종로2가'
 '태평로2가' '충정로1가' '도렴동' '봉래동1가' '을지로1가' '공평동' '백현동' '정자동' '운중동' '삼평동' '중학동'
 '남대문로1가' '남대문로2가']
타겟 동과 구 매핑 결과:
          동    구
20      청진동  종로구
29      소공동   중구
35      야탑동  분당구
36      수내동  분당구
38      서현동  분당구
43      서린동  종로구
67      구미동  분당구
86      무교동   중구
105     수하동   중구
126     금곡동  분당구
175    종로2가  종로구
212   태평로2가   중구
214   충정로1가   중구
218     도렴동  종로구
221   봉래동1가   중구
261   을지로1가   중구
264     공평동  종로구
266     백현동  분당구
315     정자동  분당구
370     운중동  분당구
373     삼평동  분당구
380     중학동  종로구
462  남대문로1가   중구
709  남대문로2가   중구
Shape before add: (43225, 6)
Shape after add: (47449, 6)


In [18]:
# 1. 모든 조합 생성
years = range(2014, 2025)
quarters = ['1Q', '2Q', '3Q', '4Q']
all_periods = [(year, quarter) for year in years for quarter in quarters]
dong_list = df6['동'].unique().tolist()
type_list = df6['건물용도'].unique().tolist()[0:4]
combination = []
for (year, quarter) in all_periods:
    for dong in dong_list:
        gu = df6.loc[df6['동'] == dong, '구'].values[0]  # 구가 존재하지 않을 때 처리
        for building_type in type_list:
            combination.append((year, quarter, dong, gu, building_type))

# 2. 기존 데이터셋에서 조합을 찾아서 없는 조합을 찾기
existing_combinations = df6[['거래년도', '거래분기', '동', '구' ,'건물용도']].apply(tuple, axis=1).tolist()
missing_combinations = [combo for combo in combination if combo not in existing_combinations]
print(f"Existing combinations: {len(existing_combinations)}")
print(f"Missing combinations: {len(missing_combinations)}")

# 누락된 조합의 중복 체크
missing_duplicates = len(missing_combinations) - len(set(missing_combinations))
print(f"Number of duplicate missing combinations: {missing_duplicates}")

# 3. 새로운 조합을 데이터프레임으로 만들고 NA 값으로 채우기
new_rows = pd.DataFrame(missing_combinations, columns=['거래년도', '거래분기', '동', '구','건물용도'])
new_rows['주거용_실거래가_건물면적당평균(만원/㎡)'] = pd.NA

df6_updated = pd.concat([df6, new_rows], ignore_index=True)
print(df6_updated.info())


Existing combinations: 47449
Missing combinations: 32467
Number of duplicate missing combinations: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79916 entries, 0 to 79915
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   거래년도                    79916 non-null  int64  
 1   거래분기                    79916 non-null  object 
 2   구                       79916 non-null  object 
 3   동                       79916 non-null  object 
 4   건물용도                    79916 non-null  object 
 5   주거용_실거래가_건물면적당평균(만원/㎡)  43225 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 3.7+ MB
None


**결측값 보간**  
1. 동, 건물용도에 따라서 보간
2. 구, 건물용도에 따라 보간 (서울)
3. 건물용도별 보간 (분당)
- 시간 보간 가능
    - 결측치 아닌행 3개 이상 & 양끝값 존재
- 선형 보간 가능
    - 결측치 아닌행 2개 이상

In [19]:
## 거래날짜 열 임의로 생성
# 거래년도와 거래분기를 결합하여 날짜로 변환하는 함수
def convert_to_date(row):
    year = row['거래년도']
    quarter = row['거래분기']
    month = (int(quarter[0]) - 1) * 3 + 1  # 1Q -> 1, 2Q -> 4, 3Q -> 7, 4Q -> 10
    return pd.Timestamp(year=year, month=month, day=1)

df6_updated['거래날짜'] = df6_updated.apply(convert_to_date, axis=1)

df6_interpolated = df6_updated.copy()

df6_interpolated['거래날짜'] = pd.to_datetime(df6_interpolated['거래년도'].astype(str) + '-01-01')
df6_interpolated.set_index('거래날짜', inplace=True)

동별, 건물용도별 보간

In [20]:
# 1단계: 동, 건물용도 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
def time_interpolate(group):
    return group.interpolate(method='time')

# 3개 이상인 그룹에 대해 시간 보간 수행
df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df6_interpolated.groupby(['동','건물용도'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 2단계: 남은 결측치 중 동, 건물용도 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
def linear_interpolate_if_enough(group):
    if len(group) == 2:
        return group.interpolate(method='linear')
    return group

# 2개 이상인 그룹에 대해 선형 보간 수행
df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df6_interpolated.groupby(['동','건물용도'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 결측치 확인
print("주거용 실거래가 결측치 수 (전체):", df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'].isnull().sum())

# 인덱스를 원래 형태로 되돌리기
df6_interpolated.reset_index(inplace=True)

주거용 실거래가 결측치 수 (전체): 19712


구별, 건물용도별 보간

In [21]:
df6_interpolated.set_index('거래날짜', inplace=True)

# 타겟 동 목록 정의
target_dongs_seoul = ['청진동', '소공동', '서린동', '무교동', '수하동', '종로2가', 
    '태평로2가', '충정로1가', '도렴동', '봉래동1가', '을지로1가', 
    '공평동', '중학동', '남대문로1가', '남대문로2가']  # 타겟 동 목록

# 1단계: 동, 구별, 건물용도, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df6_interpolated.groupby(['구','건물용도'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 인덱스를 원래대로 되돌리기
df6_interpolated.reset_index(inplace=True)

# 2단계: 남은 결측치 중 구별, 건물용도, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df6_interpolated.groupby(['구','건물용도'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 전체 결측치 확인
print("주거용실거래가 결측치 수:", df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'].isnull().sum())


주거용실거래가 결측치 수: 1584


건물용도별 보간(분당구)

In [22]:
df6_interpolated.set_index('거래날짜', inplace=True)

# 타겟 동 목록 정의
target_dongs_bundang = df5_final[df5_final['구']=='분당구']['동'].unique().tolist() # 타겟 동 목록

# 보간 함수 정의
def linear_interpolate_if_enough2(series):
    if series.count() >= 2:  # 결측치가 아닌 행이 2개 이상인 경우
        return series.interpolate(method='linear')
    else:
        return series  # 보간하지 않고 원래 시리즈 반환

# 건물용도별 보간 수행
df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df6_interpolated.groupby(['건물용도'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough2(x))
)

# 인덱스를 원래대로 되돌리기
df6_interpolated.reset_index(inplace=True)

# 전체 결측치 확인
print("주거용실거래가 결측치 수:", df6_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'].isna().sum())

df6_pivot = df6_interpolated.drop(columns=['거래날짜'])


주거용실거래가 결측치 수: 0


In [23]:
# 건물용도의 값에 따라 열 이름 변경
df6_pivot = df6_pivot.pivot_table(
    index=['거래년도', '거래분기', '동'],
    columns='건물용도',
    values='주거용_실거래가_건물면적당평균(만원/㎡)'
).reset_index()

# 열 이름을 변경하여 원하는 형식으로 만들기
df6_pivot.columns = [
    '거래년도', '거래분기', '동',
] + [f"{건물용도}_실거래가_건물면적당평균(만원/㎡)" for 건물용도 in df6_pivot.columns[3:]]

df6_pivot.head()


,거래년도,거래분기,동,단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡)
0,2014,1Q,가락동,418.816412,949.904358,511.850403,358.620450
1,2014,1Q,가리봉동,394.621898,308.476557,334.908329,306.219417
2,2014,1Q,가산동,202.523392,394.937670,300.169605,623.750787
3,2014,1Q,가양동,226.188004,496.808367,416.026241,381.207206
4,2014,1Q,가회동,1245.531398,394.547951,611.470340,655.430050


In [24]:
df6_col = pd.merge(df5_final.iloc[:,:6], df6_pivot, on=['거래년도','거래분기','동'], how='left').iloc[:,-4:]
df6_final = df5_final.copy()
df6_final['단독다가구_실거래가_건물면적당평균(만원/㎡)'] = df6_col['단독다가구_실거래가_건물면적당평균(만원/㎡)']
df6_final['아파트_실거래가_건물면적당평균(만원/㎡)'] = df6_col['아파트_실거래가_건물면적당평균(만원/㎡)']
df6_final['연립다세대_실거래가_건물면적당평균(만원/㎡)'] = df6_col['연립다세대_실거래가_건물면적당평균(만원/㎡)']
df6_final['오피스텔_실거래가_건물면적당평균(만원/㎡)'] = df6_col['오피스텔_실거래가_건물면적당평균(만원/㎡)']

df6_final.iloc[:,24:30].isna().sum()

업무용_실거래가_연면적당평균(만원/㎡)         0
단독다가구_실거래가_건물면적당평균(만원/㎡)      0
아파트_실거래가_건물면적당평균(만원/㎡)        0
연립다세대_실거래가_건물면적당평균(만원/㎡)      0
오피스텔_실거래가_건물면적당평균(만원/㎡)       0
주거용_실거래가_건물면적당평균(만원/㎡)      181
dtype: int64

# 7. 주거용_실거래가_연면적당평균(만원/㎡)
> 서울시 결측치 처리 : 대체 (보간법)
- 동별 보간: raw_data 동별 값 2개 이상 존재  
- 구별 보간: raw_data 동별 값  or 존재X 

1. 시간 보간  
 : 결측치 아닌행 3개 이상 & 양끝값 존재
2. 선형 보간  
  : 결측치 아닌행 2개 이상

> 분당구 결측치 처리 : 대체 
- 건물용도별 실거래가의 평균

In [25]:
# 1차 전처리 후, merge 이전 파일
df7 = pd.read_csv('../data/processed1_주거용_실거래가_연면적당평균.csv', encoding='cp949')
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15967 entries, 0 to 15966
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   거래년도                    15967 non-null  int64  
 1   거래분기                    15967 non-null  object 
 2   구                       15967 non-null  object 
 3   동                       15967 non-null  object 
 4   주거용_실거래가_건물면적당평균(만원/㎡)  15967 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 623.8+ KB


raw_data에 없는 거래년도,거래분기,구,동 탐색 후 생성

In [26]:
# df7['동']의 고유값
unique_df = df7['동'].unique() #428

# df6_final['동']의 고유값
unique_df_merge = df6_final['동'].unique() #188

# df6_final['동']에 속하는 값들이 df7['동']에 모두 포함되는지 확인
is_included = np.isin(unique_df_merge, unique_df)

# 결과 출력
if all(is_included):
    print('포함')
else:
    print('포함X 동 존재')
    missing_values = unique_df_merge[~is_included]
    print("포함되지 않은 값들:", missing_values)
    # raw_data에서 값이 하나도 없는 '동'

# 동 list
target_dongs = [
    '청진동', '소공동', '서린동', '무교동', '수하동', '종로2가', 
    '태평로2가', '충정로1가', '도렴동', '봉래동1가', '을지로1가', 
    '공평동', '중학동', '남대문로1가', '남대문로2가',
    '야탑동', '수내동', '서현동', '구미동', '금곡동', 
    '백현동', '정자동', '운중동', '삼평동'
] #24

# 동에 해당하는 구 가져오기
target_mapping = df6_final[df6_final['동'].isin(target_dongs)][['동', '구']].drop_duplicates()
print("타겟 동과 구 매핑 결과:")
print(target_mapping)

# 추가할 행 생성
additional_rows = []
for index, row in target_mapping.iterrows():  # target_mapping을 DataFrame으로 유지
    dong = row['동']
    gu = row['구']
    for year in range(2014, 2025):
        for quarter in ['1Q', '2Q', '3Q', '4Q']:
            additional_rows.append({
                '거래년도': year,
                '거래분기': quarter,
                '구': gu,
                '동': dong,
                '주거용_실거래가_건물면적당평균(만원/㎡)': None  # 혹시 실거래가가 없다면 None으로 설정
            })

# 추가할 데이터프레임 생성
additional_df = pd.DataFrame(additional_rows)
print('Shape before add:', df7.shape)

# 기존 df와 추가된 df 결합
df7 = pd.concat([df7, additional_df], axis=0)
print('Shape after add:', df7.shape)

포함X 동 존재
포함되지 않은 값들: ['청진동' '소공동' '야탑동' '수내동' '서현동' '서린동' '구미동' '무교동' '수하동' '금곡동' '종로2가'
 '태평로2가' '충정로1가' '도렴동' '봉래동1가' '을지로1가' '공평동' '백현동' '정자동' '운중동' '삼평동' '중학동'
 '남대문로1가' '남대문로2가']
타겟 동과 구 매핑 결과:
          동    구
20      청진동  종로구
29      소공동   중구
35      야탑동  분당구
36      수내동  분당구
38      서현동  분당구
43      서린동  종로구
67      구미동  분당구
86      무교동   중구
105     수하동   중구
126     금곡동  분당구
175    종로2가  종로구
212   태평로2가   중구
214   충정로1가   중구
218     도렴동  종로구
221   봉래동1가   중구
261   을지로1가   중구
264     공평동  종로구
266     백현동  분당구
315     정자동  분당구
370     운중동  분당구
373     삼평동  분당구
380     중학동  종로구
462  남대문로1가   중구
709  남대문로2가   중구
Shape before add: (15967, 5)
Shape after add: (17023, 5)


In [27]:
# 1. 전체 조합 생성 (거래년도, 거래분기, 동, 구)
years = range(2014, 2025)
quarters = ['1Q', '2Q', '3Q', '4Q']
all_periods = [(year, quarter) for year in years for quarter in quarters]
dong_list = df7['동'].unique()
combination = []
for (year, quarter) in all_periods:
    for dong in dong_list:
        gu = df7.loc[df7['동'] == dong, '구'].values[0]  # 동에 해당하는 구를 직접 가져옴
        combination.append((year, quarter, dong, gu))
# 전체 조합의 개수
print(f"Total combinations: {len(combination)}")

# 2. 기존 데이터셋에서 조합을 찾아서 없는 조합을 찾기
existing_combinations = df7[['거래년도', '거래분기', '동', '구']].apply(tuple, axis=1).tolist() #기존데이터
missing_combinations = [combo for combo in combination if combo not in existing_combinations] #없는 조합
print(f"Existing combinations: {len(existing_combinations)}")
print(f"Missing combinations: {len(missing_combinations)}")

# 누락된 조합의 중복 체크
missing_duplicates = len(missing_combinations) - len(set(missing_combinations))
print(f"Number of duplicate missing combinations: {missing_duplicates}")

# 3. 새로운 조합을 데이터프레임으로 만들고 NA 값으로 채우기
new_rows = pd.DataFrame(missing_combinations, columns=['거래년도', '거래분기', '동', '구'])

# 기존 데이터프레임의 업무용 실거래가 열은 그대로 두고 새로 생성된 열에 NA 할당
new_rows['주거용_실거래가_건물면적당평균(만원/㎡)'] = pd.NA

# 최종 결과물 생성: 기존 데이터프레임과 새로 생성된 행 결합
df7_updated = pd.concat([df7, new_rows], ignore_index=True)
df7_updated.info()

Total combinations: 19888
Existing combinations: 17023
Missing combinations: 2978
Number of duplicate missing combinations: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   거래년도                    20001 non-null  int64  
 1   거래분기                    20001 non-null  object 
 2   구                       20001 non-null  object 
 3   동                       20001 non-null  object 
 4   주거용_실거래가_건물면적당평균(만원/㎡)  15967 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 781.4+ KB


In [28]:
# 거래날짜 임의로 생성
def convert_to_date(row):
    year = row['거래년도']
    quarter = row['거래분기']
    month = (int(quarter[0]) - 1) * 3 + 1  # 1Q -> 1, 2Q -> 4, 3Q -> 7, 4Q -> 10
    return pd.Timestamp(year=year, month=month, day=1)

df7_updated['거래날짜'] = df7_updated.apply(convert_to_date, axis=1)
df7_updated['거래날짜'] = pd.to_datetime(df7_updated['거래날짜'])

df7_interpolated = df7_updated.copy()

df7_interpolated['거래날짜'] = pd.to_datetime(df7_interpolated['거래년도'].astype(str) + '-01-01')
df7_interpolated.set_index('거래날짜', inplace=True)
df7_interpolated.head()

,거래년도,거래분기,구,동,주거용_실거래가_건물면적당평균(만원/㎡)
거래날짜,,,,,
2014-01-01,2014,1Q,강남구,개포동,1143.856825
2014-01-01,2014,1Q,강남구,논현동,720.913872
2014-01-01,2014,1Q,강남구,대치동,1014.114945
2014-01-01,2014,1Q,강남구,도곡동,978.393028
2014-01-01,2014,1Q,강남구,삼성동,952.716791


**결측값 대체** : 서울시

In [29]:
## 동별 보간

# 1단계: 동 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
def time_interpolate(group):
    return group.interpolate(method='time')

# 3개 이상인 그룹에 대해 시간 보간 수행
df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df7_interpolated.groupby(['동'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 2단계: 남은 결측치 중 동 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
def linear_interpolate_if_enough(group):
    if len(group) == 2:
        return group.interpolate(method='linear')
    return group

# 2개 이상인 그룹에 대해 선형 보간 수행
df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df7_interpolated.groupby(['동'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 결측치 확인
print("주거용 실거래가 결측치 수 (전체):", df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'].isnull().sum())

# 인덱스를 원래 형태로 되돌리기
df7_interpolated.reset_index(inplace=True)


주거용 실거래가 결측치 수 (전체): 1056


In [30]:
## 구별 보간
df7_interpolated.set_index('거래날짜', inplace=True)

# 타겟 동 목록 정의
target_dongs = [
    '청진동', '소공동', '서린동', '무교동', '수하동', '종로2가', 
    '태평로2가', '충정로1가', '도렴동', '봉래동1가', '을지로1가', 
    '공평동', '중학동', '남대문로1가', '남대문로2가']  # 타겟 동 목록

# 1단계: 동, 구별, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 3개 이상인 경우 시간 보간 수행
df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df7_interpolated.groupby(['구'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: time_interpolate(x) if len(x) >= 3 else x)
)

# 인덱스를 원래대로 되돌리기
df7_interpolated.reset_index(inplace=True)

# 2단계: 남은 결측치 중 구별, 거래년도, 거래분기 기준으로 결측치가 아닌 행이 2개 이상인 경우 선형 보간
df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'] = (
    df7_interpolated.groupby(['구'])['주거용_실거래가_건물면적당평균(만원/㎡)']
    .transform(lambda x: linear_interpolate_if_enough(x))
)

# 전체 결측치 확인
print("주거용 실거래가 결측치 수:", df7_interpolated['주거용_실거래가_건물면적당평균(만원/㎡)'].isnull().sum())


df7_interpolated = df7_interpolated.drop(columns=['거래날짜'])
df7_interpolated[df7_interpolated.iloc[:,-1].isna()]['구'].unique()
# 분당구는 raw_data에 없어서 동별, 구별 보간으로 극복할 수 없음.

주거용 실거래가 결측치 수: 396


array(['분당구'], dtype=object)

In [31]:
df7_col = pd.merge(df6_final, df7_interpolated, on=['거래년도','거래분기','구','동'], how='left').iloc[:,-1]
df7_final = df6_final.copy()
df7_final['주거용_실거래가_건물면적당평균(만원/㎡)'] = df7_col
df7_final['주거용_실거래가_건물면적당평균(만원/㎡)'].isna().sum()

79

**결측치 보간** : 분당구

In [32]:
# '구'가 '분당구'인 경우에 한해 각 열의 평균을 계산하여 새로운 변수 추가
df7_final.loc[df7_final['구'] == '분당구', '주거용_실거래가_건물면적당평균(만원/㎡)'] = df7_final.loc[df7_final['구'] == '분당구', 
    ['단독다가구_실거래가_건물면적당평균(만원/㎡)', 
     '아파트_실거래가_건물면적당평균(만원/㎡)', 
     '연립다세대_실거래가_건물면적당평균(만원/㎡)', 
     '오피스텔_실거래가_건물면적당평균(만원/㎡)']].mean(axis=1)

# 데이터프레임 정보 확인
df7_final.iloc[:,24:30].isna().sum()

업무용_실거래가_연면적당평균(만원/㎡)       0
단독다가구_실거래가_건물면적당평균(만원/㎡)    0
아파트_실거래가_건물면적당평균(만원/㎡)      0
연립다세대_실거래가_건물면적당평균(만원/㎡)    0
오피스텔_실거래가_건물면적당평균(만원/㎡)     0
주거용_실거래가_건물면적당평균(만원/㎡)      0
dtype: int64

# 8. 상업용 임대동향 지표 (오피스_)
오피스_ 평균연면적(㎡) / 공실률(%) / 전환율(%) / 임대료(천원/㎡) / 임대수입(%) / 기타수입(%) / 영업경비(%) /   
순영업소득(%) / 순영업소득(천원/㎡)/ 소득수익률(%) / 자본수익률(%) / 투자수익률(%)

결측행 : 49rows
> 결측치 처리 : 대체

raw_data

In [33]:
상업용부동산_임대동향 = {}
for sheet_num in ['101','103','105','113','114','115']:
    df_tmp = pd.read_excel('../data/raw/24년 2분기 상업용부동산 임대동향조사 통계표(공표용).xlsx', sheet_name=sheet_num)
    상업용부동산_임대동향[df_tmp.iloc[1,0]] = df_tmp

df8 = pd.DataFrame()
for key in 상업용부동산_임대동향.keys():    
    df_tmp = 상업용부동산_임대동향[key]

    # 3번째 행을 컬럼명으로 설정, 1~3행 삭제
    df_tmp.columns = df_tmp.iloc[2]
    df_tmp = df_tmp.drop([0,1,2],axis=0).reset_index(drop=True)
    
    #' (1) 투자수익률' 시트에만 '상세코드'와 '항목CODE'가 있어 오류나는 걸 해결
    try:
        df_tmp = df_tmp.drop(['지역CODE','상세코드','항목CODE'],axis=1)
    except:
        df_tmp = df_tmp.drop(['지역CODE'],axis=1)
    
    # 서울과 분당만 남김
    df_tmp = df_tmp[(df_tmp['지역구분(1)'] == '서울')|(df_tmp['지역구분(2)'] == '분당역세권')]
        
    df8 = pd.concat([df8,df_tmp],axis=0)

# 2014년 이후 데이터만 남김
df8.drop(['2013.1Q','2013.2Q','2013.3Q','2013.4Q'], axis=1, inplace=True)
df8 = df8.sort_values(by='항목').reset_index(drop=True)

**결측치 대체**

In [34]:
print('기존 결측치 수: ',df8.isna().sum().sum())
row_결측 = df8.loc[df8['2014.1Q'].isna(),['지역구분(2)','항목']]

# 결측인 지역과, 항목을 하나씩 for문 실행
for idx, row in row_결측.iterrows():
    지역 = row['지역구분(2)']
    항목 = row['항목']

    # 결측인 행 추출
    tmp_row = df8[(df8['항목'] == 항목) & (df8['지역구분(2)'] == 지역)]

    # 상관계수를 비교할, 결측이 아닌 열이름 추출
    col_not_NA = tmp_row.columns[tmp_row.iloc[0,:].notna()][3:]
    col_NA = tmp_row.columns[tmp_row.iloc[0,:].isna()]


    # 해당되는 항목의 지역 행 추출
    df_corr = df8.loc[df8['항목'] == 항목]

    # index를 지역구분(2)로 변경
    df_corr.index = df_corr['지역구분(2)']
    df_corr = df_corr[col_not_NA]
    df_corr = df_corr[col_not_NA].T.corr()
    # 상관계수 계산 후 절대값을 취하고, 해당 지역만 추출, 결측이 존재했던 지역은 제외, 값 sort
    corr_list = abs(df_corr[지역]).drop(row_결측['지역구분(2)'].unique(),axis=0).sort_values(ascending=False)

    # # 본인 제외 상관계수가 가장 높은 지역과 값 추출
    best_corr = corr_list[0]
    best_loc = corr_list.index[0]
    
    if pd.isna(best_corr):
        unique_num = float(tmp_row[col_not_NA[0]].values)
        print(f'\"{지역}\"의 {항목} 값은 모두 {unique_num}으로 같은 값이라 correlation 산출 불가. 결측치를 {unique_num}으로 대체.')
        df8.loc[(df8['항목'] == 항목) & (df8['지역구분(2)'] == 지역),col_NA] = unique_num
    else:            
        print(f'\"{지역}\"의 {항목}와(과) correlation이 가장 높은 지역은 \"{best_loc}\" : {np.round(best_corr,4)}')

        # 상관계수가 가장 높은 지역의 데이터 추출
        best_loc_data = df8.loc[(df8['지역구분(2)'] == best_loc) & (df8['항목'] == 항목)]

        # 결측치를 다음 시점의 값을 기준으로 상관계수가 높은 지역의 추세를 반영하여 뒤에서 부터 채우기
        for col in col_NA[::-1]:
            next_col = tmp_row.columns[tmp_row.columns.get_loc(col) + 1]  # 다음 시점
            
            # 상관계수가 높은 지역의 다음 시점 값과 현재 시점 값
            current_val = best_loc_data[col]
            next_val = best_loc_data[next_col]
            
            # 다음 시점의 값에 상관계수 높은 지역의 변동률을 적용하여 결측치 채움
            df8.loc[(df8['항목'] == 항목) & (df8['지역구분(2)'] == 지역), col] = \
                float(df8.loc[(df8['항목'] == 항목) & (df8['지역구분(2)'] == 지역), next_col].values * (current_val / next_val))
print('\n처리 후 결측치 수: ',df8.isna().sum().sum())


기존 결측치 수:  3864
"잠실/송파"의 공실률(%)와(과) correlation이 가장 높은 지역은 "소계(기타지역)" : 0.7272
"용산역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "분당역세권" : 0.9013
"숙명여대"의 공실률(%)와(과) correlation이 가장 높은 지역은 "계" : 0.8978
"을지로"의 공실률(%)와(과) correlation이 가장 높은 지역은 "소계(도심지역)" : 0.9306
"영등포역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "광화문" : 0.6677
"당산역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "신사역" : 0.7167
"논현역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "남대문" : 0.6995
"남부터미널"의 공실률(%)와(과) correlation이 가장 높은 지역은 "테헤란로" : 0.7439
"시청"의 공실률(%)와(과) correlation이 가장 높은 지역은 "소계(도심지역)" : 0.8622
"교대역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "분당역세권" : 0.6875
"잠실새내역"의 공실률(%)와(과) correlation이 가장 높은 지역은 "광화문" : 0.6041
"교대역"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "종로" : 0.8343
"당산역"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "소계(강남지역)" : 0.6344
"남부터미널"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "소계(강남지역)" : 0.418
"논현역"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "공덕역" : 0.9145
"영등포역"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "천호" : 0.7871
"시청"의 기타수입(%)와(과) correlation이 가장 높은 지역은 "공덕역" :

형태변환 후 merge

In [35]:
# 지역 - 항목 - 기간 - 값 형태로 melt
df8_melt = pd.melt(df8, id_vars=['지역구분(1)','지역구분(2)','항목'], var_name='기간', value_name='값')

# 기간을 거래년도와 거래분기로 분리
df8_melt['거래년도'] = df8_melt['기간'].apply(lambda x: x[:4]).astype(int)
df8_melt['거래분기'] = df8_melt['기간'].apply(lambda x: x[5:])

# 컬럼 순서 재정렬
df8_melt = df8_melt[['거래년도','거래분기','지역구분(1)','지역구분(2)','항목','값']]

# 행 재정렬
df8_melt = df8_melt.sort_values(by=['거래년도','거래분기','지역구분(1)','지역구분(2)']).reset_index(drop=True)

print('기간당 데이터 종류 수:',len(df8_melt['항목'].unique()))
print('기간당 데이터 종류:', df8_melt['항목'].unique())
display(df8_melt.head(15))

# 지역 매칭 필요
print('**지역구분**\n',df8_melt['지역구분(2)'].unique())

# 변수명 설정을 위해 항목 앞에 '오피스_' 추가
df8_melt['항목'] = df8_melt['항목'].apply(lambda x: '오피스_'+x)

기간당 데이터 종류 수: 14
기간당 데이터 종류: ['공실률(%)' '기타수입(%)' '동수(동)' '소득수익률(%)' '순영업소득(%)' '순영업소득(천원/㎡)' '영업경비(%)'
 '임대료(천원/㎡)' '임대수입(%)' '자본수익률(%)' '전환율(%)' '투자수익률(%)' '평균연면적(㎡)' '평균층수(층)']


,거래년도,거래분기,지역구분(1),지역구분(2),항목,값
0,2014,1Q,경기,분당역세권,공실률(%),21.5
1,2014,1Q,경기,분당역세권,기타수입(%),0.577956
2,2014,1Q,경기,분당역세권,동수(동),6
3,2014,1Q,경기,분당역세권,소득수익률(%),1.692942
4,2014,1Q,경기,분당역세권,순영업소득(%),70.132352
5,2014,1Q,경기,분당역세권,순영업소득(천원/㎡),36
6,2014,1Q,경기,분당역세권,영업경비(%),29.867648
7,2014,1Q,경기,분당역세권,임대료(천원/㎡),16.516719
8,2014,1Q,경기,분당역세권,임대수입(%),99.422044
9,2014,1Q,경기,분당역세권,자본수익률(%),-0.329497


**지역구분**
 ['분당역세권' '강남대로' '계' '공덕역' '광화문' '교대역' '남대문' '남부터미널' '논현역' '당산역' '도산대로'
 '동대문' '명동' '목동' '사당' '소계(강남지역)' '소계(기타지역)' '소계(도심지역)' '소계(여의도마포지역)'
 '숙명여대' '시청' '신사역' '여의도' '영등포역' '용산역' '을지로' '잠실/송파' '잠실새내역' '장안동' '종로'
 '천호' '충무로' '테헤란로' '홍대/합정' '화곡']


In [36]:
# 항목과 값을 cast
df8_filtered = df8_melt.pivot_table(index=['거래년도', '거래분기', '지역구분(1)', '지역구분(2)'], columns='항목', values='값').reset_index()

# 사용할 열만 추출
df8_col = df8_filtered[['거래년도', '거래분기', '지역구분(2)', 
                       '오피스_평균연면적(㎡)','오피스_공실률(%)','오피스_전환율(%)','오피스_임대료(천원/㎡)',
                       '오피스_임대수입(%)','오피스_기타수입(%)','오피스_영업경비(%)','오피스_순영업소득(%)', '오피스_순영업소득(천원/㎡)',
                       '오피스_소득수익률(%)','오피스_자본수익률(%)', '오피스_투자수익률(%)']]
df8_col.columns = ['거래년도', '거래분기', '지역구분(2)', 
                   '오피스_평균연면적(㎡)','오피스_공실률(%)','오피스_전환율(%)','오피스_임대료(천원/㎡)',
                   '오피스_임대수입(%)','오피스_기타수입(%)','오피스_영업경비(%)','오피스_순영업소득률(%)', '오피스_순영업소득(천원/㎡)',
                   '오피스_소득수익률(%)','오피스_자본수익률(%)', '오피스_투자수익률(%)']
df8_col.rename(columns={'지역구분(2)': '권역2'}, inplace=True)
df8_col.columns.name = None
df8_col.head()

,거래년도,거래분기,권역2,오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득률(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
0,2014,1Q,분당역세권,16729.0,21.5,12.0,16.516719,99.422044,0.577956,29.867648,70.132352,36.0,1.692942,-0.329497,1.363445
1,2014,1Q,강남대로,8970.0,12.8,12.4,23.854682,99.728242,0.271758,19.561652,80.438348,68.0,0.966286,0.248596,1.214881
2,2014,1Q,계,9266.0,8.9,11.8,20.441567,98.909512,1.090488,23.798965,76.201035,53.0,1.391067,0.184811,1.575878
3,2014,1Q,공덕역,20721.0,3.2,10.2,16.369239,99.144424,0.855576,31.969302,68.030698,46.0,1.566998,-0.053708,1.51329
4,2014,1Q,광화문,15512.0,7.8,11.5,21.307219,99.053264,0.946736,26.098656,73.901344,55.0,1.609872,0.052319,1.662191


In [37]:
# 상업용 임대동향 지표(오피스_) : 권역2, 거래년도, 거래분기
df8_result = df7_final.iloc[:,:30]

# 권역2 분류기준
categories = {
    '분당역세권': ['야탑동', '수내동', '서현동', '정자동'],
    '강남대로': ['논현동', '신사동', '삼성동', '청담동', '역삼동', '대치동', '서초동', '양재동'],
    '공덕역': ['공덕동', '도화동'],
    '광화문': ['서린동', '청진동', '종로2가', '종로6가'],
    '교대역': ['서초동'],
    '남대문': ['남대문로1가', '남대문로2가', '남대문로5가'],
    '남부터미널': ['서초동', '방배동'],
    '논현역': ['논현동'],
    '당산역': ['당산동1가', '당산동3가', '당산동4가', '당산동5가', '당산동6가'],
    '도산대로': ['신사동', '청담동'],
    '동대문': ['제기동', '휘경동', '청량리동'],
    '명동': ['명동1가', '필동1가', '필동2가', '필동3가'],
    '목동': ['목동'],
    '사당': ['사당동'],
    '숙명여대': ['남영동'],
    '시청': ['무교동', '태평로2가'],
    '신사역': ['신사동'],
    '여의도': ['여의도동'],
    '영등포역': ['영등포동2가', '영등포동4가', '영등포동6가', '영등포동7가'],
    '용산역': ['한강로2가', '한강로3가', '후암동'],
    '을지로': ['을지로1가', '을지로2가', '을지로3가', '을지로4가', '을지로6가'],
    '잠실/송파': ['잠실동', '신천동', '석촌동', '풍납동', '송파동', '오금동'],
    '잠실새내역': ['신천동'],
    '장안동': ['장안동'],
    '종로': ['서린동', '청진동', '종로2가', '종로6가'],
    '천호': ['천호동'],
    '충무로': ['충무로1가', '충무로3가'],
    '테헤란로': ['삼성동', '역삼동'],
    '홍대/합정': ['서교동', '합정동', '동교동'],
    '화곡': ['화곡동']
}

# 각 동을 권역2 기준으로 재분류
def classify_dong(dong):
    for category, dongs in categories.items():
        if dong in dongs:
            return category
    return '소계(기타지역)'

# 새로운 열 '권역2' 추가
df8_result['권역2'] = df8_result['동'].apply(classify_dong)
df8_result['권역2'].value_counts(dropna=False)
print("권역2 추가 후 shape:",df8_result.shape)

# 상업용 임대동향 지표(오피스_) merge - 거래년도, 거래분기, 권역2
df8_final = pd.merge(df8_result, df8_col, on=['거래년도','거래분기','권역2'], how='left')
df8_final.drop(columns='권역2', inplace=True)
print("권역2 제거 후 shape:",df8_final.shape)
print(df8_final.columns)
print(df8_final.isna().sum())
print(df8_final.info())
display(df8_final.head())
display(df8_final.tail())


권역2 추가 후 shape: (968, 31)
권역2 제거 후 shape: (968, 42)
Index(['거래년도', '거래분기', '시', '구', '동', '거래당시연면적', '거래당시토지면적', '빌딩비고', '거래비고',
       '거래가', '지상층', '지하층', '준공년도', '건물유형', '권역', '기준금리(%)', 'APT매매가격지수',
       '오피스매매가격지수', '소비자물가지수', '건설_소계_건설공사비지수', '주거용건물_건설공사비지수',
       '비주거용건물_건설공사비지수', '건축보수_건설공사비지수', '표준지공시지가평균(원/㎡)',
       '업무용_실거래가_연면적당평균(만원/㎡)', '단독다가구_실거래가_건물면적당평균(만원/㎡)',
       '아파트_실거래가_건물면적당평균(만원/㎡)', '연립다세대_실거래가_건물면적당평균(만원/㎡)',
       '오피스텔_실거래가_건물면적당평균(만원/㎡)', '주거용_실거래가_건물면적당평균(만원/㎡)', '오피스_평균연면적(㎡)',
       '오피스_공실률(%)', '오피스_전환율(%)', '오피스_임대료(천원/㎡)', '오피스_임대수입(%)',
       '오피스_기타수입(%)', '오피스_영업경비(%)', '오피스_순영업소득률(%)', '오피스_순영업소득(천원/㎡)',
       '오피스_소득수익률(%)', '오피스_자본수익률(%)', '오피스_투자수익률(%)'],
      dtype='object')
거래년도                        0
거래분기                        0
시                           0
구                           0
동                           0
거래당시연면적                     0
거래당시토지면적                    0
빌딩비고                        0
거래비고              

,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역,기준금리(%),APT매매가격지수,오피스매매가격지수,소비자물가지수,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수,표준지공시지가평균(원/㎡),업무용_실거래가_연면적당평균(만원/㎡),단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡),오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득률(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
0,2014,1Q,서울,영등포구,여의도동,27202.05,3602.0,일반,일반거래,14600,12.0,4.0,1985.0,집합,여의도권,2.5,51.681705,55.544697,92.885,84.696667,83.91,84.67,84.73,4097.330378,419.924977,332.585402,724.699407,636.464075,532.679833,680.294881,23158.0,8.8,12.0,20.607946,98.591924,1.408076,25.107995,74.892005,54.0,1.496556,0.275278,1.771833
1,2014,1Q,서울,강남구,논현동,3516.43,713.6,일반,일반거래,8100,7.0,2.0,2004.0,일반,강남권,2.5,56.321412,50.959516,92.885,84.696667,83.91,84.67,84.73,22213.120943,416.498724,713.731962,799.790201,614.019391,663.488648,720.913872,8970.0,12.8,12.4,23.854682,99.728242,0.271758,19.561652,80.438348,68.0,0.966286,0.248596,1.214881
2,2014,1Q,서울,서대문구,충정로3가,25984.49,2705.0,일반,일반거래,13900,17.0,4.0,1995.0,일반,기타권,2.5,63.183296,55.021148,92.885,84.696667,83.91,84.67,84.73,32572.694881,65.659910,220.187262,511.106102,454.674188,477.732794,494.933719,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884
3,2014,1Q,서울,서대문구,충정로3가,31894.78,3772.0,일반,일반거래,10500,17.0,4.0,1987.0,집합,기타권,2.5,63.183296,55.021148,92.885,84.696667,83.91,84.67,84.73,32572.694881,65.659910,220.187262,511.106102,454.674188,477.732794,494.933719,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884
4,2014,1Q,서울,용산구,한남동,4370.31,730.5,일반,일반거래,25200,5.0,4.0,2012.0,일반,기타권,2.5,58.804654,55.021148,92.885,84.696667,83.91,84.67,84.73,23992.170118,185.353208,744.626378,1090.179995,758.559705,793.650794,979.598932,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884


,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역,기준금리(%),APT매매가격지수,오피스매매가격지수,소비자물가지수,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수,표준지공시지가평균(원/㎡),업무용_실거래가_연면적당평균(만원/㎡),단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡),오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득률(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
963,2024,2Q,서울,서초구,서초동,29784.83,4588.60,일반,일반거래,22500,14.0,5.0,1990.0,일반,강남권,3.5,96.355817,101.007436,113.32,130.13,129.043333,129.69,128.7,42315.651531,932.250811,943.637360,2577.502357,1229.101160,951.826774,1980.716163,11827.024264,9.903127,8.785714,28.870845,99.796874,0.203126,13.284225,86.715775,84.101704,0.821505,2.313352,3.134857
964,2024,2Q,서울,구로구,구로동,3556.72,819.70,일반,일반거래,12100,7.0,2.0,1990.0,일반,영등포구로권,3.5,87.266501,101.944307,113.32,130.13,129.043333,129.69,128.7,33337.713799,310.419288,402.366416,923.598932,800.187427,551.722568,845.662296,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
965,2024,2Q,서울,영등포구,대림동,5464.08,1216.00,일반,일반거래,13300,10.0,2.0,1991.0,일반,영등포구로권,3.5,93.494579,101.944307,113.32,130.13,129.043333,129.69,128.7,26763.324530,365.979207,515.173578,1091.962677,641.216749,480.800654,902.726075,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
966,2024,2Q,서울,광진구,자양동,4104.82,1541.40,일반,일반거래,14900,5.0,1.0,1990.0,일반,기타권,3.5,93.925438,101.961123,113.32,130.13,129.043333,129.69,128.7,34846.450652,450.689677,1147.189061,1563.983856,1313.816038,1399.228517,1401.634106,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
967,2024,2Q,서울,동대문구,신설동,7247.65,804.51,일반,일반거래,19600,14.0,3.0,2008.0,일반,기타권,3.5,86.398296,101.961123,113.32,130.13,129.043333,129.69,128.7,28296.555528,913.908290,528.841193,956.343334,1435.678158,857.147025,1031.261256,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107


# + 건물연식 
변수 추가 
- 건물연식 = 2024-준공년도

In [38]:
df_final = df8_final.copy()
# 건물연식 추가
df_final['건물연식'] = 2024 - df_final['준공년도']
df_final.columns

Index(['거래년도', '거래분기', '시', '구', '동', '거래당시연면적', '거래당시토지면적', '빌딩비고', '거래비고',
       '거래가', '지상층', '지하층', '준공년도', '건물유형', '권역', '기준금리(%)', 'APT매매가격지수',
       '오피스매매가격지수', '소비자물가지수', '건설_소계_건설공사비지수', '주거용건물_건설공사비지수',
       '비주거용건물_건설공사비지수', '건축보수_건설공사비지수', '표준지공시지가평균(원/㎡)',
       '업무용_실거래가_연면적당평균(만원/㎡)', '단독다가구_실거래가_건물면적당평균(만원/㎡)',
       '아파트_실거래가_건물면적당평균(만원/㎡)', '연립다세대_실거래가_건물면적당평균(만원/㎡)',
       '오피스텔_실거래가_건물면적당평균(만원/㎡)', '주거용_실거래가_건물면적당평균(만원/㎡)', '오피스_평균연면적(㎡)',
       '오피스_공실률(%)', '오피스_전환율(%)', '오피스_임대료(천원/㎡)', '오피스_임대수입(%)',
       '오피스_기타수입(%)', '오피스_영업경비(%)', '오피스_순영업소득률(%)', '오피스_순영업소득(천원/㎡)',
       '오피스_소득수익률(%)', '오피스_자본수익률(%)', '오피스_투자수익률(%)', '건물연식'],
      dtype='object')

# + 변수명 정제
변수명 뒤의 괄호 및 특수기호 삭제

In [39]:
# 괄호 및 괄호 안의 내용을 맨 뒤에서만 삭제하는 함수
def remove_trailing_parentheses(variable_name):
    return re.sub(r'\s*\(.*?\)$', '', variable_name)  # 괄호와 그 안의 내용을 맨 뒤에서 삭제

# 모든 변수명에 대해 함수 적용
df_final.columns = [remove_trailing_parentheses(col) for col in df_final.columns]

# 결측치 삭제 확인

In [40]:
# 각 열의 결측치 수 계산
na_counts = df_final.iloc[:,1:].isnull().sum()
na_df = pd.DataFrame({'Column': na_counts.index, 'NA_Count': na_counts.values})
num_rows = (len(na_df) +3- 2) // 3  # 전체 행 수 계산
na_df_gathered = pd.DataFrame(columns=['Column1', 'NA_Count1', 'Column2', 'NA_Count2', 'Column3', 'NA_Count3'])
for i in range(num_rows):
    row_data = []
    for j in range(3):
        index = i + j * num_rows
        if index < len(na_df):
            row_data.append(na_df.iloc[index, 0])  # Column
            row_data.append(na_df.iloc[index, 1])  # NA_Count
        else:
            row_data.extend([None, None])  # 남는 부분은 None으로 채우기
    na_df_gathered.loc[i] = row_data
na_df_gathered

,Column1,NA_Count1,Column2,NA_Count2,Column3,NA_Count3
0,거래분기,0,기준금리,0,주거용_실거래가_건물면적당평균,0
1,시,0,APT매매가격지수,0,오피스_평균연면적,0
2,구,0,오피스매매가격지수,0,오피스_공실률,0
3,동,0,소비자물가지수,0,오피스_전환율,0
4,거래당시연면적,0,건설_소계_건설공사비지수,0,오피스_임대료,0
5,거래당시토지면적,0,주거용건물_건설공사비지수,0,오피스_임대수입,0
6,빌딩비고,0,비주거용건물_건설공사비지수,0,오피스_기타수입,0
7,거래비고,0,건축보수_건설공사비지수,0,오피스_영업경비,0
8,거래가,0,표준지공시지가평균,0,오피스_순영업소득률,0
9,지상층,0,업무용_실거래가_연면적당평균,0,오피스_순영업소득,0


In [41]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   거래년도                968 non-null    int64  
 1   거래분기                968 non-null    object 
 2   시                   968 non-null    object 
 3   구                   968 non-null    object 
 4   동                   968 non-null    object 
 5   거래당시연면적             968 non-null    float64
 6   거래당시토지면적            968 non-null    float64
 7   빌딩비고                968 non-null    object 
 8   거래비고                968 non-null    object 
 9   거래가                 968 non-null    int64  
 10  지상층                 968 non-null    float64
 11  지하층                 968 non-null    float64
 12  준공년도                968 non-null    float64
 13  건물유형                968 non-null    object 
 14  권역                  968 non-null    object 
 15  기준금리                968 non-null    float64
 16  APT매매가격지

In [42]:
# csv로 저장
df_final.to_csv('../data/processed2_merged.csv',index=False, encoding='cp949')